In [40]:
from dotenv import load_dotenv
load_dotenv()
cohere_api_key=os.getenv("COHERE_API_KEY")
alpha_vantage_api_key=os.getenv("ALPHA_VANTAGE_API_KEY")

In [54]:
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.llms import Cohere
import requests

def fetch_financial_data(api_key, ticker):
    """
    Fetch financial data for the given ticker using Alpha Vantage API.
    Returns a dictionary with expanded financial metrics.
    """
    base_url = "https://www.alphavantage.co/query"
    income_params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": api_key
    }
    balance_params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": api_key
    }
    cash_flow_params = {
        "function": "CASH_FLOW",
        "symbol": ticker,
        "apikey": api_key
    }

    # Fetch Income Statement
    income_response = requests.get(base_url, params=income_params)
    if income_response.status_code != 200:
        return {"error": "Failed to fetch income statement data"}
    income_data = income_response.json()

    # Fetch Balance Sheet
    balance_response = requests.get(base_url, params=balance_params)
    if balance_response.status_code != 200:
        return {"error": "Failed to fetch balance sheet data"}
    balance_data = balance_response.json()

    # Fetch Cash Flow Statement
    cash_flow_response = requests.get(base_url, params=cash_flow_params)
    if cash_flow_response.status_code != 200:
        return {"error": "Failed to fetch cash flow data"}
    cash_flow_data = cash_flow_response.json()

    # Extract relevant data
    try:
        latest_income = income_data["annualReports"][0]
        latest_balance = balance_data["annualReports"][0]
        latest_cash_flow = cash_flow_data["annualReports"][0]
        
        return {
            "revenue": float(latest_income["totalRevenue"]),
            "net_income": float(latest_income["netIncome"]),
            "interest_expense": (latest_income.get("interestExpense", 0)),
            "total_liabilities": float(latest_balance["totalLiabilities"]),
            "total_debt": float(latest_balance.get("totalDebt", 0)),
            "total_equity": float(latest_balance.get("totalShareholderEquity", 0)),
            "current_assets": float(latest_balance.get("totalCurrentAssets", 0)),
            "current_liabilities": float(latest_balance.get("totalCurrentLiabilities", 0)),
            "inventory": float(latest_balance.get("inventory", 0)),
            "cash_and_equivalents": float(latest_balance.get("cashAndCashEquivalentsAtCarryingValue", 0)),
            "total_assets": float(latest_balance["totalAssets"]),
            "operating_cash_flow": float(latest_cash_flow.get("operatingCashflow", 0))
        }
    except (KeyError, IndexError):
        return {"error": "Incomplete financial data"}

def compute_ratios(data):
    """
    Compute enhanced financial ratios from the provided data.
    Handles potential division by zero scenarios.
    """
    # Extract values with safe defaults
    revenue = data.get("revenue", 1)  # Avoid division by zero
    net_income = data.get("net_income", 0)
    total_liabilities = data.get("total_liabilities", 0)
    total_debt = data.get("total_debt", 0)
    total_equity = data.get("total_equity", 1)
    current_assets = data.get("current_assets", 0)
    current_liabilities = data.get("current_liabilities", 1)
    inventory = data.get("inventory", 0)
    cash_and_equivalents = data.get("cash_and_equivalents", 0)
    total_assets = data.get("total_assets", 1)
    interest_expense = data.get("interest_expense", 0)
    operating_cash_flow = data.get("operating_cash_flow", 0)

    # Compute ratios with error handling
    ratios = {
        # Existing Ratios
        "Return on Equity (ROE)": net_income / total_equity if total_equity != 0 else 0,
        "Return on Assets (ROA)": net_income / total_assets if total_assets != 0 else 0,
        "Gross Margin": (revenue - total_liabilities) / revenue if revenue else 0,
        "Net Profit Margin": net_income / revenue if revenue else 0,
        
        # New Financial Ratios
        "Debt to Equity Ratio": total_debt / total_equity if total_equity != 0 else float('inf'),
        
        "Interest Coverage Ratio": (net_income + interest_expense) / interest_expense if interest_expense != 0 else float('inf'),
        
        "Current Ratio": current_assets / current_liabilities if current_liabilities != 0 else float('inf'),
        
        "Quick Ratio": (current_assets - inventory) / current_liabilities if current_liabilities != 0 else float('inf'),
        
        "Inventory Turnover": revenue / inventory if inventory != 0 else float('inf'),
        
        "Asset Turnover": revenue / total_assets if total_assets != 0 else float('inf')
    }
    
    return ratios

def fetch_financial_tool(input_text):
    """
    Fetch and calculate financial ratios based on stock ticker input.
    """
    ticker = input_text.upper()
    # api_key = "YOUR_ALPHA_VANTAGE_API_KEY"  # Replace with your Alpha Vantage API key
    data = fetch_financial_data(alpha_vantage_api_key, ticker)
    if "error" in data:
        return data["error"]
    ratios = compute_ratios(data)
    return ratios

tools = [
    Tool(
        name="Fetch Financial Ratios",
        func=fetch_financial_tool,
        description="Fetches financial data and computes comprehensive ratios based on a stock ticker."
    )
]

# Cohere LLM setup
cohere_llm = Cohere(model="command-xlarge")  # Replace with your Cohere API key

from langchain.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are a financial assistant. Use the tools provided to answer questions.

Tools available:
{tools}

Names of tools: {tool_names}

Conversation history:
{history}

Current user input:
{input}

Assistant's reasoning and actions:
{agent_scratchpad}
"""
)

# Create the agent
memory = ConversationBufferMemory(memory_key="history")
agent = create_structured_chat_agent(
    llm=cohere_llm,
    tools=tools,
    prompt=prompt,
    output_key="output"
)

# Create executor
executor = AgentExecutor(agent=agent, tools=tools)

ticker = "AAPL"  # Replace with desired stock ticker
print(executor.run(f"Fetch financial ratios for {ticker}"))


TypeError: create_structured_chat_agent() got an unexpected keyword argument 'output_key'

In [55]:
fetch_financial_tool(ticker)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [29]:
main()

Financial Analysis for AAPL:
Ticker: AAPL
Return on Equity (ROE): 164.59%
Return on Assets (ROA): 25.68%
Gross Margin: 46.21%
Net Profit Margin: 23.97%
Debt to Equity Ratio: 5.41
